# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ffa486b6370>
├── 'a' --> tensor([[-0.0867, -2.1062,  0.4195],
│                   [-0.7969, -0.4119,  0.1763]])
└── 'x' --> <FastTreeValue 0x7ffa486b6430>
    └── 'c' --> tensor([[ 0.7013,  0.2276, -0.9438,  0.0500],
                        [-1.1480,  1.3692,  0.3580, -0.6541],
                        [ 1.2554,  0.6613,  0.3205, -0.6622]])

In [4]:
t.a

tensor([[-0.0867, -2.1062,  0.4195],
        [-0.7969, -0.4119,  0.1763]])

In [5]:
%timeit t.a

66.8 ns ± 1.33 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ffa486b6370>
├── 'a' --> tensor([[-0.6857,  0.0506,  1.1968],
│                   [ 0.2954, -1.4878, -0.7920]])
└── 'x' --> <FastTreeValue 0x7ffa486b6430>
    └── 'c' --> tensor([[ 0.7013,  0.2276, -0.9438,  0.0500],
                        [-1.1480,  1.3692,  0.3580, -0.6541],
                        [ 1.2554,  0.6613,  0.3205, -0.6622]])

In [7]:
%timeit t.a = new_value

66.2 ns ± 1.36 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0867, -2.1062,  0.4195],
               [-0.7969, -0.4119,  0.1763]]),
    x: Batch(
           c: tensor([[ 0.7013,  0.2276, -0.9438,  0.0500],
                      [-1.1480,  1.3692,  0.3580, -0.6541],
                      [ 1.2554,  0.6613,  0.3205, -0.6622]]),
       ),
)

In [10]:
b.a

tensor([[-0.0867, -2.1062,  0.4195],
        [-0.7969, -0.4119,  0.1763]])

In [11]:
%timeit b.a

73.8 ns ± 1.75 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2828,  0.3061,  0.2805],
               [ 0.2765, -0.7611, -0.1266]]),
    x: Batch(
           c: tensor([[ 0.7013,  0.2276, -0.9438,  0.0500],
                      [-1.1480,  1.3692,  0.3580, -0.6541],
                      [ 1.2554,  0.6613,  0.3205, -0.6622]]),
       ),
)

In [13]:
%timeit b.a = new_value

590 ns ± 12.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.02 µs ± 18.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.2 µs ± 90.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

264 µs ± 17.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

239 µs ± 3.92 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff9992f7b20>
├── 'a' --> tensor([[[-0.0867, -2.1062,  0.4195],
│                    [-0.7969, -0.4119,  0.1763]],
│           
│                   [[-0.0867, -2.1062,  0.4195],
│                    [-0.7969, -0.4119,  0.1763]],
│           
│                   [[-0.0867, -2.1062,  0.4195],
│                    [-0.7969, -0.4119,  0.1763]],
│           
│                   [[-0.0867, -2.1062,  0.4195],
│                    [-0.7969, -0.4119,  0.1763]],
│           
│                   [[-0.0867, -2.1062,  0.4195],
│                    [-0.7969, -0.4119,  0.1763]],
│           
│                   [[-0.0867, -2.1062,  0.4195],
│                    [-0.7969, -0.4119,  0.1763]],
│           
│                   [[-0.0867, -2.1062,  0.4195],
│                    [-0.7969, -0.4119,  0.1763]],
│           
│                   [[-0.0867, -2.1062,  0.4195],
│                    [-0.7969, -0.4119,  0.1763]]])
└── 'x' --> <FastTreeValue 0x7ff9992f7c70>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

45.8 µs ± 1.4 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff999307040>
├── 'a' --> tensor([[-0.0867, -2.1062,  0.4195],
│                   [-0.7969, -0.4119,  0.1763],
│                   [-0.0867, -2.1062,  0.4195],
│                   [-0.7969, -0.4119,  0.1763],
│                   [-0.0867, -2.1062,  0.4195],
│                   [-0.7969, -0.4119,  0.1763],
│                   [-0.0867, -2.1062,  0.4195],
│                   [-0.7969, -0.4119,  0.1763],
│                   [-0.0867, -2.1062,  0.4195],
│                   [-0.7969, -0.4119,  0.1763],
│                   [-0.0867, -2.1062,  0.4195],
│                   [-0.7969, -0.4119,  0.1763],
│                   [-0.0867, -2.1062,  0.4195],
│                   [-0.7969, -0.4119,  0.1763],
│                   [-0.0867, -2.1062,  0.4195],
│                   [-0.7969, -0.4119,  0.1763]])
└── 'x' --> <FastTreeValue 0x7ff9a01569d0>
    └── 'c' --> tensor([[ 0.7013,  0.2276, -0.9438,  0.0500],
                        [-1.1480,  1.3692,  0.3580, -0.6541],
                 

In [23]:
%timeit t_cat(trees)

40.8 µs ± 561 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

77 µs ± 929 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.0867, -2.1062,  0.4195],
                [-0.7969, -0.4119,  0.1763]],
       
               [[-0.0867, -2.1062,  0.4195],
                [-0.7969, -0.4119,  0.1763]],
       
               [[-0.0867, -2.1062,  0.4195],
                [-0.7969, -0.4119,  0.1763]],
       
               [[-0.0867, -2.1062,  0.4195],
                [-0.7969, -0.4119,  0.1763]],
       
               [[-0.0867, -2.1062,  0.4195],
                [-0.7969, -0.4119,  0.1763]],
       
               [[-0.0867, -2.1062,  0.4195],
                [-0.7969, -0.4119,  0.1763]],
       
               [[-0.0867, -2.1062,  0.4195],
                [-0.7969, -0.4119,  0.1763]],
       
               [[-0.0867, -2.1062,  0.4195],
                [-0.7969, -0.4119,  0.1763]]]),
    x: Batch(
           c: tensor([[[ 0.7013,  0.2276, -0.9438,  0.0500],
                       [-1.1480,  1.3692,  0.3580, -0.6541],
                       [ 1.2554,  0.6613,  0.3205, -0.6622]],
         

In [26]:
%timeit Batch.stack(batches)

96.4 µs ± 1.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.0867, -2.1062,  0.4195],
               [-0.7969, -0.4119,  0.1763],
               [-0.0867, -2.1062,  0.4195],
               [-0.7969, -0.4119,  0.1763],
               [-0.0867, -2.1062,  0.4195],
               [-0.7969, -0.4119,  0.1763],
               [-0.0867, -2.1062,  0.4195],
               [-0.7969, -0.4119,  0.1763],
               [-0.0867, -2.1062,  0.4195],
               [-0.7969, -0.4119,  0.1763],
               [-0.0867, -2.1062,  0.4195],
               [-0.7969, -0.4119,  0.1763],
               [-0.0867, -2.1062,  0.4195],
               [-0.7969, -0.4119,  0.1763],
               [-0.0867, -2.1062,  0.4195],
               [-0.7969, -0.4119,  0.1763]]),
    x: Batch(
           c: tensor([[ 0.7013,  0.2276, -0.9438,  0.0500],
                      [-1.1480,  1.3692,  0.3580, -0.6541],
                      [ 1.2554,  0.6613,  0.3205, -0.6622],
                      [ 0.7013,  0.2276, -0.9438,  0.0500],
                      [-1.1480,  

In [28]:
%timeit Batch.cat(batches)

172 µs ± 2.12 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

492 µs ± 8.95 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
